In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import gsw

import cognac.ufloat as fl

---

## 09/2020 balastage cuve acoustique

In [2]:
root_path = '/Users/aponte/Current_projects/cognac/tests/202011_ifremer/'

# to turn
store = False

### v1

In [3]:
bfile = root_path+'balast_v1.yaml'

b = fl.balast()

# ipressure = 640, batteries = 11.14
b.add_balast('cuve_acoustique-02/11/2020', 
             mass_in_water=421.6,
             mass_in_air=13633.8, 
             water_temperature=20.8,
             water_salinity=0.5,
             piston_displacement='max',
            )
#
if store:
    b.store(bfile)
    
# reload
b1 = fl.balast(bfile)

File /Users/aponte/Current_projects/cognac/tests/202011_ifremer/balast_v1.yaml loaded


### v2

In [4]:
bfile = root_path+'balast_v2.yaml'

b = fl.balast()

# ipressure = 640, batteries = 11.14
b.add_balast('cuve_acoustique-02/11/2020', 
             mass_in_water=435.8,
             mass_in_air=13729, 
             water_temperature=20.8,
             water_salinity=0.5,
             piston_displacement='max',
            )
#
if store:
    b.store(bfile)
    
# reload
b2 = fl.balast(bfile)

File /Users/aponte/Current_projects/cognac/tests/202011_ifremer/balast_v2.yaml loaded


---

## compute mass adjustments


In [5]:
# init float
f = fl.autonomous_float(model='Ifremer')
f.init_piston()

Piston max volume set from max displacement


In [11]:
#lon, lat = '4W30.', '48N00.' # Brest
lon, lat = '6W00.', '42N30.' # Med Toulon

t, s = 14.5, 37.3
b1.compute_mass_adjustment(f, temperature=t, salinity=s, lon=lon, lat=lat,)

t, s = 19., 37.
b1.compute_mass_adjustment(f, temperature=t, salinity=s, lon=lon, lat=lat,)

New in situ density: 1027.86 kg/m3
-- According to balast cuve_acoustique-02/11/2020, you need add -38.2 g
(if the weight is external to the float, remember this must be the value in water)
New in situ density: 1026.56 kg/m3
-- According to balast cuve_acoustique-02/11/2020, you need add -51.1 g
(if the weight is external to the float, remember this must be the value in water)


In [7]:
b2.compute_mass_adjustment(f, temperature=t, salinity=s, lon=lon, lat=lat,)

New in situ density: 1027.86 kg/m3
-- According to balast cuve_acoustique-02/11/2020, you need add -50.0 g
(if the weight is external to the float, remember this must be the value in water)


In [21]:
add = 158 # 156, 158, 160 in water
float_plus_add = 94
add - float_plus_add

64

----

## Equations and uncertainties


### step 1: computation of the volume in a controled environment

In order to estimate the volume of the float, one must weigh it in air (mass $m$) and under water (mass $m^{water}$).
The target precision is high (typically gramms) and this can only be done in a controled environment (still water, known temperature, salinity) with accurate tools (temperature, salinity sensors, balance).
Precisions required for this instruments will be specified next.

The mass of the float underwater is given by:

$
\begin{align}
m^{water} &=  (\rho-\rho_{water})V \\
&= m - \rho_{water} V,
\end{align}
$

where $\rho = m/V$ is the float density.
This leads to:

$
\begin{align}
V &= \frac{m-m^{water}}{\rho_{water}} 
\end{align}
$


An additional mass ($m_{add}^{water}$ measured underwater) may have to be added in order to submerge the float.
The mass of the float underwater is then related to the total weight under water $m_{float+add}^{water}$ by:

$
\begin{align}
m^{water} &= m_{float+add}^{water} - m_{add}^{water}
\end{align}
$

Volume uncertainties associated with unaccurate masses, temperature, salinity are given by:

$
\begin{align}
e_{m} &= \delta m /\rho_{water}, \\
e_{T} &= V \times \frac{\partial_T\rho_{water}}{\rho_{water}}  \delta T = V \times \alpha \delta T, \\
e_{S} &= V \times \frac{\partial_S\rho_{water}}{\rho_{water}}  \delta S = V \times \beta \delta S.
\end{align}
$

For $\delta_m = 1$g, $\delta T=0.1$degC, $\delta S = 0.1$psu, $V=12\times 10^3$cm^3 (see cell below):

$
\begin{align}
e_{m} &\sim 1.0\;cm^3 \\
e_{T} &\sim 0.3\;cm^3, \\
e_{S} &\sim 0.9\;cm^3.
\end{align}
$

In [8]:
# select a water profile
w = fl.waterp(lon=6.,lat=40.) # Mediterranean sea
z=np.arange(-100.,0.,1.)

gamma = w.get_compressibility(z).mean()
alpha = w.get_alpha(z).mean()
beta = w.get_beta(z).mean()

print('gamma_water = {:.3} /dbar'.format(gamma) )
print('alpha_water = {:.3} /degC'.format(alpha) )
print('beta_water = {:.3} /psu'.format(beta) )

V = 12*1e3 # cm^3
dm = 1 # g
dT = .1 # degC
dS = .1 # psu
rho_water = 1030 # kg

print('e_m = {:.3} cm3'.format(dm*1e-3/rho_water*1e6) )
print('e_T = {:.3} cm3'.format(V*alpha*dT) )
print('e_S = {:.3} cm3'.format(V*beta*dS) )

gamma_water = 4.24e-06 /dbar
alpha_water = 0.000231 /degC
beta_water = 0.000738 /psu
e_m = 0.971 cm3
e_T = 0.277 cm3
e_S = 0.886 cm3


---

### step 2: adjustment of the mass predeployment

We compute the mass adjustement required with the following equations.

$
\begin{align}
\delta m^{water} = 0 &= \delta m - V \delta \rho_{water} - \rho_{water} \delta V,
\end{align}
$

$\delta m$ is the mass that needs to be added or substracted and is therefore given by:

$
\begin{align}
\delta m  &= V \delta \rho_{water} + \rho_{water} \delta V,
\end{align}
$

Note that unlike step 1, this step takes into account the float thermal compressibility via $\delta V$.

Any uncertainties on water temperature/salinity will induce uncertainties comparable to that of step 1:

In [10]:
print('e_T = {:.3} g'.format(V*alpha*rho_water/1e3*dT) )
print('e_S = {:.3} g'.format(V*beta*rho_water/1e3*dS) )

e_T = 0.286 g
e_S = 0.913 g
